In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def try_open_file(file_path, encodings=['utf-8', 'latin-1', 'iso-8859-1']):
    """Tries to open a file with multiple encodings."""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, sep=';', encoding=encoding, on_bad_lines='warn')
            return df, encoding
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode file with available encodings: {file_path}")

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            try:
                df, used_encoding = try_open_file(os.path.join(folder_path, file_name))
                dataframes[file_name] = df
                print(f"Loaded {file_name} with encoding {used_encoding}")
            except Exception as e:
                print(f"Failed to load {file_name}: {e}")
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    
    if 'classification_cleaned' in df.columns:
        df = df[df['classification_cleaned'].isin(valid_classes) & df['classification_cleaned'].notna()]
        discarded_rows = initial_rows - df.shape[0]
    else:
        discarded_rows = initial_rows
    
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    
    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0
    skipped_files = []

    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns:
            skipped_files.append(file_name)
            continue
        try:
            df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
            common_index = df_pred_clean.index.intersection(ground_truth_df.index)
            y_pred = df_pred_clean.loc[common_index, 'classification_cleaned']
            y_true_common = y_true.loc[common_index]
            
            metrics = calculate_metrics(y_true_common, y_pred)
            metrics['file_name'] = file_name
            results.append(metrics)
            
            total_discarded_rows += discarded_rows
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    metrics_df = pd.DataFrame(results)

    if skipped_files:
        print("\nSkipped files:")
        for file_name in skipped_files:
            print(file_name)

    return metrics_df, total_discarded_rows

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = r'C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = r"C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\MIXTRAL\csv_from_log_files\manually_checked_and_correct"

metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)
print(metrics_df)
print(f"Total discarded rows: {total_discarded_rows}")


Loaded _EN_reply_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_classify_replies_561_MIXTRAL_Hatebase_FINAL_FOURTH.csv with encoding utf-8
Failed to load _EN_reply_classify_replies_561_MIXTRAL_Hatebase_FINAL_SECOND.csv: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.

Loaded _EN_reply_classify_replies_561_MIXTRAL_Hatebase_FINAL_THIRD.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_MIXTRAL_Hatebase_FINAL_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_title_definition_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_title_descriptionYT_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_title_descriptio

C:\Users\aboeck\AppData\Local\Temp\ipykernel_13120\3470190754.py:22: ParserWarning: Skipping line 16: expected 3 fields, saw 6
Skipping line 17: expected 3 fields, saw 6
Skipping line 20: expected 3 fields, saw 4
Skipping line 25: expected 3 fields, saw 5
Skipping line 66: expected 3 fields, saw 4
Skipping line 90: expected 3 fields, saw 6

  df = pd.read_csv(file_path, sep=';', encoding=encoding, on_bad_lines='warn')
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div


Skipped files:
_EN_reply_comment_title_transcript_object_classify_replies_561_MIXTRAL_Hatebase_FINAL - SECOND.csv
    accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
0   0.600719         0.282222      0.222078  0.213074            0.623421   
1   0.590991         0.345238      0.220925  0.215011            0.692321   
2   0.590991         0.345238      0.220925  0.215011            0.692321   
3   0.618280         0.294403      0.282240  0.232015            0.631790   
4   0.612208         0.313080      0.285343  0.244646            0.661177   
5   0.594255         0.291823      0.276743  0.235127            0.631201   
6   0.513761         0.266730      0.254809  0.254458            0.600660   
7   0.599641         0.317071      0.325864  0.237632            0.667179   
8   0.497248         0.283957      0.236862  0.203137            0.622582   
9   0.509091         0.270322      0.437791  0.213695            0.599563   
10  0.589792         0.261381      0.2

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [2]:
metrics_df

accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
0   0.600719         0.282222      0.222078  0.213074            0.623421   
1   0.590991         0.345238      0.220925  0.215011            0.692321   
2   0.590991         0.345238      0.220925  0.215011            0.692321   
3   0.618280         0.294403      0.282240  0.232015            0.631790   
4   0.612208         0.313080      0.285343  0.244646            0.661177   
5   0.594255         0.291823      0.276743  0.235127            0.631201   
6   0.513761         0.266730      0.254809  0.254458            0.600660   
7   0.599641         0.317071      0.325864  0.237632            0.667179   
8   0.497248         0.283957      0.236862  0.203137            0.622582   
9   0.509091         0.270322      0.437791  0.213695            0.599563   
10  0.589792         0.261381      0.223517  0.216492            0.585765   
11  0.537634         0.323739      0.303305  0.225508            0.681502   
12  0.588454         0.246288      0.271206  0.219576            0.561389   
13  0.608303         0.246456      0.275358  0.221770            0.577455   
14  0.598182         0.294120      0.375744  0.240038            0.617935   
15  0.595668         0.269042      0.219717  0.208577            0.603458   
16  0.612613         0.328369      0.277616  0.225453            0.675541   

    recall_weighted  f1_weighted  \
0          0.600719     0.563790   
1          0.590991     0.558363   
2          0.590991     0.558363   
3          0.618280     0.579070   
4          0.612208     0.594729   
5          0.594255     0.577701   
6          0.513761     0.544282   
7          0.599641     0.582759   
8          0.497248     0.517487   
9          0.509091     0.518152   
10         0.589792     0.554033   
11         0.537634     0.557266   
12         0.588454     0.548579   
13         0.608303     0.567816   
14         0.598182     0.564330   
15         0.595668     0.554804   
16         0.612613     0.573999   

                                            file_name  
0   _EN_reply_classify_replies_561_MIXTRAL_Hatebas...  
1   _EN_reply_classify_replies_561_MIXTRAL_Hatebas...  
2   _EN_reply_classify_replies_561_MIXTRAL_Hatebas...  
3   _EN_reply_comment_classify_replies_561_MIXTRAL...  
4   _EN_reply_comment_title_classify_replies_561_M...  
5   _EN_reply_comment_title_classify_replies_561_M...  
6   _EN_reply_comment_title_definition_classify_re...  
7   _EN_reply_comment_title_descriptionYT_classify...  
8   _EN_reply_comment_title_description_classify_r...  
9   _EN_reply_comment_title_description_classify_r...  
10  _EN_reply_comment_title_external_context_class...  
11  _EN_reply_comment_title_metadata_video_classif...  
12  _EN_reply_comment_title_metadata_video_classif...  
13  _EN_reply_comment_title_transcript_classify_re...  
14  _EN_reply_comment_title_transcript_description...  
15  _EN_reply_comment_title_transcript_object_clas...  
16  _EN_reply_title_classify_replies_561_MIXTRAL_H...

In [3]:
metrics_df.to_excel("RESULTS_EXPERIMENTS_MIXTRAL.xlsx")